# Create mass balance tables for PPP-fate based on the pppfate-observer in beecs_ecotox

This notebook takes data from 100 model runs for 4 different model versions, namely the basic beecs or beecs_ecotox version "baseline", the same version with just the thermoregulation fix applied, then the same version with thermoregulation and the nursefactor fix applied, and lastly the same version with all mentioned fixes and the HSU fix applied.

The observer "PPPFateObs" observs all possible PPP sinks the model has to offer. The function below takes the last day of each of the 100 runs per model version and adds these values up to a total amount of PPP that has been foraged for and then has ended up in each of the respective sinks within the model. The function is quite chaotic and not very clear, maybe I will add some comments and structure still; but basically it just takes these summed up metrics for PPP fate of the model and calculates the fractions of PPP that went into specific endpoints within the model to then create a sort of massbalance. It calculates two possible losses of PPP, once outside of the hive for PPP that gets foraged but does not reach the in-hive stores and once for PPP within the hive that has reached the stores but disappears somewhere along the way.

In [ ]:
import numpy as np
import pandas as pd
from os import path

# read in all of the excel sheets and aggregate the respective amounts over all 100 runs
# make sure the files actually exist and change the path if necessary
# if the error "NameError: name 'null' is not defined" occurs, this is almost certainly the case because the files do either not exist
# or cannot be found by path. Make sure the directory and file names work out and try to open the CSV-file in another tab; for some reason this fixes the issue sometimes

file_names = ["baseline", "thermofix", "nursefix", "HSUfix"]
dir = "Out/"

def agg_to_table(dir, filenames):
    tab = pd.DataFrame() 

    for filename in filenames:
        file_pattern = dir + filename + "-%04d.csv"
        results = pd.DataFrame()
        idx = 0
        while True:
            file = file_pattern % (idx,)
            if not path.exists(file):
                break

            run = pd.read_csv(file, delimiter=";")
            if results.empty:
                results = run.iloc[[-1]]  
            else:
                results = pd.concat([results, run.iloc[[-1]]], ignore_index=True) 

            idx += 1

        sums = []
        columns = list(results.columns)[1:]

        for col in columns:
            sums.append(results[col].sum())
        
        lostPPP = sums[0] - sums[1] - sums[2] - sums[3] - sums[4]
        sums.append(lostPPP)

        totalPPPinhive = sums[1] + sums[2]    
        sums.append(totalPPPinhive)

        lostPPP = totalPPPinhive - sums[5] - sums[7] - sums[8] - sums[9] - sums[10] - sums[11] - sums[14]
        sums.append(lostPPP)

        temp_tab = pd.DataFrame({  
            'model': [filename],
            'TotalPPPmass': np.round(sums[0], 2),
            columns[0]: np.round(sums[0]/sums[0] * 100, 2),
            columns[1]: np.round(sums[1]/sums[0] * 100, 2),
            columns[2]: np.round(sums[2]/sums[0] * 100, 2),
            columns[3]: np.round(sums[3]/sums[0] * 100, 2),
            columns[4]: np.round(sums[4]/sums[0] * 100, 2),
            'LostOutsideHive': np.round(sums[-3]/sums[0] * 100, 2),
            #'TotalPPPmassInHive': np.round(sums[-2], 2),
            columns[5]: np.round(sums[5]/sums[0] * 100, 2),
            columns[6]: np.round(sums[6]/sums[0] * 100, 2),
            columns[7]: np.round(sums[7]/sums[0] * 100, 2),
            columns[8]: np.round(sums[8]/sums[0] * 100, 2),
            columns[9]: np.round(sums[9]/sums[0] * 100, 2),
            columns[10]: np.round(sums[10]/sums[0] * 100, 2),
            columns[11]: np.round(sums[11]/sums[0] * 100, 2),
            columns[12]: np.round(sums[12]/sums[0] * 100, 2),
            columns[13]: np.round(sums[13]/sums[0] * 100, 2),
            columns[14]: np.round(sums[14]/sums[0] * 100, 2),
            'TotalPPPinhive': np.round(sums[-2]/sums[0] * 100, 2),
            'LostInsideHive': np.round(sums[-1]/sums[0] * 100, 2),
        })
        
        tab = pd.concat([tab, temp_tab], ignore_index=True) 

    return tab

tab = agg_to_table(dir, file_names)
toprint = tab.T


with open('massBalance_beecs.tex','w') as tf:
    tf.write(toprint.to_latex(float_format="%.2f"))

toprint

,0,1,2,3
model,baseline,thermofix,nursefix,HSUfix
TotalPPPmass,3353308.98,3341197.81,3328368.16,3356353.17
TotalPPForaged,100.0,100.0,100.0,100.0
PPPhoneyStores,38.51,38.55,38.28,38.12
PPPpollenStores,51.27,51.19,51.51,51.69
PPPforagersImmediate,4.82,4.83,4.8,9.01
PPPforagerDiedinFlight,1.12,1.15,1.16,1.18
LostOutsideHive,4.28,4.28,4.25,-0.0
PPPforagersinHive,24.57,20.31,20.24,20.21
PPPforagersTotal,29.4,25.14,25.04,29.23
